In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install tensorflow==1.6.0

In [3]:
pip install keras==2.1.5

In [4]:
import tensorflow
tensorflow.keras.__version__


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

'2.1.3-tf'

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import random
random.seed(30)

In [0]:
dir_path = '/content/drive/My Drive/CS 280/dataset-resized'

classes = ('cardboard','glass', 'metal', 'paper', 'plastic', 'trash')
num_classes = 6

In [7]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
train_datagen=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1)
val_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator=train_datagen.flow_from_directory(dir_path,
                                                  target_size=(224,224),
                                                  batch_size=128,
                                                  class_mode='categorical',
                                                  subset='training',
                                                  seed = 30)

test_generator=train_datagen.flow_from_directory(dir_path,
                                                  target_size=(224,224),
                                                  batch_size=128,
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  seed = 30)

val_generator=val_datagen.flow_from_directory(dir_path,
                                              target_size=(224,224),
                                              batch_size=128,
                                              class_mode='categorical',
                                              subset='validation',
                                              seed = 30)

Using TensorFlow backend.


Found 2293 images belonging to 6 classes.
Found 252 images belonging to 6 classes.
Found 252 images belonging to 6 classes.


In [0]:
# val = [next(test_generator) for i in range(0,5)] 
# fig, ax = plt.subplots(1,5, figsize=(15, 6)) 
# print('Labels:', [item[1][0] for item in val])
# l = [ax[i].imshow(val[i][0][0]) for i in range(0,5)]

# val = [next(val_generator) for i in range(0,5)] 
# fig, ax = plt.subplots(1,5, figsize=(15, 6)) 
# print('Labels:', [item[1][0] for item in val])
# l = [ax[i].imshow(val[i][0][0]) for i in range(0,5)]

In [9]:
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Activation, BatchNormalization, AveragePooling2D
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import backend as K

K.set_learning_phase(1)
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape= (224,224,3), classes=6)

for layer in resnet_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
  
resnet_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [10]:
model = Sequential()
model.add(resnet_model)

# Classification layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))

# Softmax
model.add(Dense(6, activation='softmax'))

# model.compile(loss='categorical_crossentropy',
#               optimizer=tensorflow.keras.optimizers.RMSprop(lr=1e-4),
#               metrics=['acc'])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])


# SVM
# from keras.regularizers import l2
# model.add(Dense(6, activation='linear', kernel_regularizer=tf.keras.regularizers.l2()))
# model.compile(loss='hinge',
#               optimizer='adadelta',
#               metrics=['acc'])


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 6150      
Total params: 25,692,038
Trainable params: 2,104,326
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
# Load model
model.load_weights('/content/drive/My Drive/CS 280/restnet50_SVM.h5')
model

In [0]:
from keras.callbacks import ModelCheckpoint

filepath='/content/drive/My Drive/CS 280/restnet50.h5'
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')
callbacks_list = [checkpoint1]



history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=50,
      validation_data=val_generator,
      validation_steps=val_generator.samples/val_generator.batch_size,
      verbose=1,
      workers=4,
      callbacks=callbacks_list)

Epoch 1/50
17/17 [===========================>..]17/17 [===========================>..] - ETA: 36s - loss: 0.5441 - acc: 0.8185 
Epoch 00001: saving model to /content/drive/My Drive/CS 280/restnet50.h5
18/17 [==============================]18/17 [==============================] - 797s 44s/step - loss: 0.5310 - acc: 0.8228 - val_loss: 0.4480 - val_acc: 0.8333

Epoch 2/50
17/17 [===========================>..]17/17 [===========================>..] - ETA: 36s - loss: 0.2096 - acc: 0.9398 
Epoch 00002: saving model to /content/drive/My Drive/CS 280/restnet50.h5
18/17 [==============================]18/17 [==============================] - 787s 44s/step - loss: 0.2151 - acc: 0.9375 - val_loss: 0.3594 - val_acc: 0.8770

Epoch 3/50
17/17 [===========================>..]17/17 [===========================>..] - ETA: 36s - loss: 0.1590 - acc: 0.9568 
Epoch 00003: saving model to /content/drive/My Drive/CS 280/restnet50.h5
18/17 [==============================]18/17 [=============================

In [0]:
model.save('/content/drive/My Drive/CS 280/restnet50.h5')